In [56]:
import pandas as pd 
import numpy as np
import pickle
import os

import datetime

from scipy import sparse 
from operator import itemgetter
from math import * 

import pymongo
local_mongo = pymongo.MongoClient("mongodb://localhost:27017/")
guidy_db = local_mongo["guidy"]
place_rating_collections = guidy_db["user_rating"]


res_data = []
for row in place_rating_collections.find({"type": "RESTAURANT"}):
    data = [row['User_Id'], row['Place_Id'], row['Rating'], row['Rating_Space'], row['Rating_Location'], row['Rating_Quality'], row['Rating_Service'], row['Rating_Price'], row['TotalLike']]
    res_data.append(data)



In [57]:
res_data = np.array(res_data)

In [13]:
time_period = []
for row in place_rating_collections.find({"type": "RESTAURANT"}):
    data = [row['TimeStamp']]
    time_period.append(data)

In [55]:
restaurant_data[0]

array([1.94069380e+07, 7.65331000e+05, 1.00000000e+01, 1.00000000e+01,
       1.00000000e+01, 1.00000000e+01, 1.00000000e+01, 1.00000000e+01,
       0.00000000e+00, 1.58402292e+09])

In [23]:
go = []
for i in range (len(time_period)):
    go.append((np.datetime64(time_period[i][0]) - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's'))

/home/yntn/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  This is separate from the ipykernel package so we can avoid doing imports until


In [58]:
restaurant_data = np.c_[res_data, np.array(go)]

In [59]:
restaurant_data

array([[1.94069380e+07, 7.65331000e+05, 1.00000000e+01, ...,
        1.00000000e+01, 0.00000000e+00, 1.58402292e+09],
       [9.31753600e+06, 7.65331000e+05, 8.00000000e+00, ...,
        8.00000000e+00, 6.00000000e+00, 1.58204735e+09],
       [1.32633700e+06, 7.68300000e+05, 7.40000000e+00, ...,
        7.00000000e+00, 0.00000000e+00, 1.54574786e+09],
       ...,
       [2.80640000e+04, 6.98934000e+05, 9.00000000e+00, ...,
        1.00000000e+01, 0.00000000e+00, 1.56310575e+09],
       [1.80586700e+06, 6.98934000e+05, 8.40000000e+00, ...,
        8.00000000e+00, 1.00000000e+00, 1.56326492e+09],
       [2.27152000e+05, 6.98934000e+05, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 1.58400163e+09]])

In [46]:
userId = list(set(restaurant_data[:,0]))
userId.sort()
mapUserId  = {}
for i, value in enumerate(userId):
    mapUserId[value] = i

placeId = list(set(restaurant_data[:,1]))
placeId.sort()
mapPlaceId  = {}
for i, value in enumerate(placeId):
    mapPlaceId[value] = i

In [47]:
for i in range(len(restaurant_data[:,0])):
    restaurant_data[i][0] = mapUserId[restaurant_data [i][0]]
for i in range(len(restaurant_data[:,1])):
    restaurant_data[i][1] = mapPlaceId[restaurant_data [i][1]]

tmp_based_copy = restaurant_data

tmp_based_copy = tmp_based_copy[tmp_based_copy[:,0].argsort()] # sort theo  user_id
based_train = []
based_test = []
_usersId = (tmp_based_copy[:, 0]).astype(np.int32)

for i in range((max(tmp_based_copy[:,0])).astype(int)):  
    _ids = np.where(_usersId == i)[0].astype(np.int32)
    if (len(_ids) > 4):
        arr = tmp_based_copy[_ids]
        arr = arr[np.argsort(arr[:, 8])]
        _down = floor(len(arr)*0.2)
        _up = len(arr)
        X_train_test = np.split(arr.tolist(), [_down, _up])
        ts = max (X_train_test[1][:,8])
        for j in range (len(X_train_test[1])):
            X_train_test[1][j,8] = (ts - X_train_test[1][j,8]) / 86400 / 90

        based_test += X_train_test[0].tolist()
        based_train += X_train_test[1].tolist()
    
based_train = np.array(based_train)
based_test = np.array(based_test)

In [48]:
tmp_based_copy[0]

array([0.00000000e+00, 3.02100000e+03, 8.40000000e+00, 9.00000000e+00,
       6.00000000e+00, 9.00000000e+00, 8.00000000e+00, 1.00000000e+01,
       0.00000000e+00, 1.40003619e+09])